In [1]:
import json
import sqlite3
import traceback

import zhipuai

from prepare_doc_tree.doc_tree import DocTree, doc_tree_search
from prepare_question.utils import get_llm_classification, get_llm_sql, get_similarity_column_name, execute_sql, answer_normalize
from utils import find_most_related_file

d:\anaconda3\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.7.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


bin d:\anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


In [11]:
txt_file_directory = r"D:\Code\DataSet\chatglm_llm_fintech_raw_dataset\alltxt"

In [12]:
question_file_list = [
    "./prepare_question/初赛/test_questions.json",
    "./prepare_question/复赛A/A-list-question.json",
    "./prepare_question/复赛B/B-list-question.json",
    "./prepare_question/复赛C/C-list-question.json"
]


In [13]:
# 创建连接
conn = sqlite3.connect("./prepare_data/company.db")
cursor = conn.cursor()

In [14]:
def request_llm(message):
    response = zhipuai.model_api.invoke(
        model="chatglm_pro",
        prompt=[
            {"role": "user", "content": message},
        ],
        temperature=1.0,
        ref={"enable": False}
    )
    result = response["data"]["choices"][0]["content"]
    return result

In [15]:
def process_question(q):
    classification = get_llm_classification(q)
    print(f"question_classification: {classification}")
    if classification["type"] == "3-1":
        file_path = find_most_related_file(q, txt_file_directory)
        dt = DocTree(file_path[0][1])
        message = "你是一个能够解读上市公司年报的智能机器人，了解大量的专业金融和商业术语，能够进行深度金融分析。"
        message += "请你根据以下提供的材料来回答最后的问题，并且答案应该尽量简洁。\n\n"
        for keyword in classification["keyword"]:
            contexts = doc_tree_search(dt, keyword)
            for context in contexts:
                message += context
        message += f"\n问题：{q}"
        return request_llm(message)
    elif classification["type"] == "3-2":
        return request_llm(q)
    else:
        cs = [col for keyword in classification["keyword"] for col in get_similarity_column_name(keyword)]
        sql = get_llm_sql(q, classification["year"], cs)
        print(f"question_sql: {sql}")
        json_data = execute_sql(cursor, sql)
        print(f"json_data: {json_data}")
        return answer_normalize(q, json_data)

In [16]:
for idx, question_file in enumerate(question_file_list):
    print(f"start to process {question_file}")
    answer_list = []
    with open(question_file, "r", encoding="utf-8") as f:
        for i, line in enumerate(f.readlines()):
            question_dict = json.loads(line)
            question = question_dict["question"]
            retry = 0
            while retry < 3:
                try:
                    print(f"question: {question}")
                    answer = process_question(question)
                    print(f"question_answer: {answer}\n")
                    question_dict["answer"] = answer.strip('"').replace("\n", "")
                    break
                except Exception as e:
                    print(f"error: {e}")
                    question_dict["answer"] = ""
                    traceback.print_exc()
                retry += 1
            answer_list.append(question_dict)

    json.dump(answer_list, open(f"{idx}-answer_list.json", 'w', encoding="utf-8"), ensure_ascii=False, indent=4)

start to process ./prepare_question/初赛/test_questions.json
question: 能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？
Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "d:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_37824\2251048942.py", line 12, in <module>
    answer = process_question(question)
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_37824\1090359555.py", line 2, in process_question
    classification = get_llm_classification(q)
  File "C:\Users\admin\Desktop\大模型实战：AIGC技术原理及实际应用\6-ChatPDF\prepare_question\utils.py", line 44, in get_llm_classification
    response = zhipuai.model_api.invoke(
  File "d:\anaconda3\lib\site-packages\zhipuai\model_api\api.py", line 20, in invoke
    return post(url, cls._generate_token(), kwargs, zhipuai.api_ti

TypeError: object of type 'NoneType' has no len()

In [30]:
# 定义源文件和目标文件的路径
source_file_path = "01-answer_list.json"
target_file_path = "0-answer_list.json"

# 使用GBK编码读取文件
with open(source_file_path, "r", encoding="utf-8") as file:
    content = json.load(file)

In [32]:
content[0]


{'id': 0,
 'question': '能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？',
 'answer': '" 2020 年，金宇生物技术股份有限公司积极履行社会责任，主要体现在以下几个方面：\\n\\n1. 防疫物资捐赠：公司向内蒙古医科大学附属医院等单位捐赠价值 30 多万元的医用口罩、防护服、护目镜等防疫物资，并向内蒙古自治区红十字会捐赠价值 1008 万元的新型冠状病毒核酸检测试剂盒，用于支持疫情的防控救治工作。\\n\\n2. 扶贫工作：公司赴清水河五良太乡白旗窑村开展党建扶贫结对共建，捐赠饲料粉加工机 2 台，价值 13600 元；为对口扶贫点清水河白旗窑村养殖户提供防疫知识分享，免费为全村 4000 只羊注射口蹄疫疫苗，开展特色扶贫。\\n\\n3. 国际援助：公司向柬埔寨捐赠牛出血性败血病灭活疫苗 10 万头份，为中柬企业在生物制药领域深度合作奠定基础。\\n\\n4. 人才培养：子公司金宇保灵以校企合作形式开展“新型学徒制培训”，协助地区稳定就业，提升人才创新技能。\\n\\n5. 关爱员工：公司工会及时了解困难员工的生活状况，并进行针对性帮扶。2020 年公司党委、工会系统帮扶困难员工 9 人，帮扶金额共计 10 万元。同时，企业组织献爱心活动，为员工家属筹集爱心捐款 10.62 万元。\\n\\n综上，金宇生物技术股份有限公司在 2020 年报告期内积极履行社会责任，关注防疫、扶贫、人才培养、员工关爱等方面的工作。"'}

In [37]:
with open(target_file_path, "w", encoding="utf-8") as file:
    for item in content:
        if not item.get("answer", None):
            try:
                question = item["question"]
                print(f"question: {question}")
                answer = process_question(question)
                print(f"question_answer: {answer}\n")
                item["answer"] = answer.strip('"').replace("\n", "")
            except Exception as e:
                print(f"error: {e}")
                item["answer"] = ""
        file.write(json.dumps(item, ensure_ascii=False) + "\n")


question: 大有能源在2021年的每股经营现金流量是多少元？
question_classification: {'company': '大有能源', 'year': [2021], 'keyword': ['每股经营现金流量'], 'formula': False, 'type': '1'}
question_sql: select 公司名称,股票简称,报告年份,经营活动产生的现金流量净额/股票数量 as 每股经营现金流量 from company where 报告年份 in ('2021') and (公司名称 in ('大有能源') or 股票简称 in ('大有能源'));
error: no such column: 股票数量
question: 力芯微在2021年的每股经营现金流量是多少元？
question_classification: {'company': '力芯微', 'year': [2021], 'keyword': ['每股经营现金流量'], 'formula': False, 'type': '1'}
error: _log() got an unexpected keyword argument 'exc_info'
question: 臻镭科技2021年的每股经营现金流量是多少元？
question_classification: {'company': '臻镭科技', 'year': [2021], 'keyword': ['每股经营现金流量'], 'formula': False, 'type': '1'}
question_sql: select 公司名称,股票简称,报告年份,经营活动产生的现金流量净额/股票总数 as 每股经营现金流量 from company where 报告年份 in ('2021') and (公司名称 in ('臻镭科技') or 股票简称 in ('臻镭科技'));
error: no such column: 股票总数
question: 清研环境2021年每股经营现金流量是多少元?
question_classification: {'company': '清研环境', 'year': [2021], 'keyword': ['每股经营现金流量'], 'formula': False, 

d:\anaconda3\lib\site-packages\langchain\schema\vectorstore.py:259: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='十五、公司股权激励计划、员工持股计划或其他员工激励措施的实施情况', metadata={'id': 180}), -104.42250425922711)]
  warnings.warn(
d:\anaconda3\lib\site-packages\langchain\schema\vectorstore.py:259: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='十五、公司股权激励计划、员工持股计划或其他员工激励措施的实施情况', metadata={'id': 180}), -123.51446287841642)]
  warnings.warn(
d:\anaconda3\lib\site-packages\langchain\schema\vectorstore.py:259: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='十五、公司股权激励计划、员工持股计划或其他员工激励措施的实施情况', metadata={'id': 180}), -107.6472763012723)]
  warnings.warn(


question_answer: " 根据提供的信息，华数传媒控股股份有限公司在 2020 年报告期内，没有实施股权激励计划、员工持股计划或其他员工激励措施。因此，对这些计划的具体分析无法进行。"

question: 上海步科自动化股份有限公司2020年的综合收益总额是多少元？
question_classification: {'company': '上海步科自动化股份有限公司', 'year': [2020], 'keyword': ['综合收益总额'], 'formula': False, 'type': '1'}
question_sql: select 公司名称,股票简称,报告年份,综合收益总额 from company where 报告年份 in ('2020') and (公司名称 in ('上海步科自动化股份有限公司') or 股票简称 in ('上海步科自动化股份有限公司'));
json_data: []
question_answer: 抱歉,根据您提供的查询结果,我无法为您提供上海步科自动化股份有限公司 2020 年的综合收益总额。请您提供更详细的信息或者重新查询相关数据。

question: 应收票据的到期和违约对公司的现金流和信用风险有何影响？
question_classification: {'company': '', 'year': [], 'keyword': ['应收票据', '到期', '违约', '现金流', '信用风险'], 'formula': False, 'type': '3-2'}
question_answer: " 应收票据是一种商业信用工具，是企业销售商品或提供劳务而收到的、由购货方签发并承诺在将来一定时间内支付货款的票据。应收票据的到期和违约都会对公司的现金流和信用风险产生影响。\n\n应收票据到期对现金流的影响:\n\n当应收票据到期时，如果公司已经提前计划并安排好了相应的资金，那么公司可以按时收回应收票据款项，从而增加公司的现金流。但如果公司没有提前计划或安排好相应的资金，那么可能会出现资金短缺，无法按时收回应收票据款项，从而影响公司的现金流。\n\n应收票据违约对信用风险的影响:\n\n当购货方无法按照约定的时间支付应收票据款项时，就发生了违约。这会对公司的信用风险产生影响。如果购货方违约，公司

d:\anaconda3\lib\site-packages\langchain\schema\vectorstore.py:259: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='三、报告期内核心竞争力分析', metadata={'id': 53}), -140.87312729497341)]
  warnings.warn(


question_answer: " 陕西广电网络传媒（集团）股份有限公司在 2020 年的核心竞争力主要体现在以下四个方面：\n\n1. 网络优势：公司覆盖陕西全省所有市、县和乡镇，形成陕北、陕南、关中三大环网，有线无线融合的网络规模为公司拓展业务奠定了坚实的网络基础。\n\n2. 政策优势：作为陕西省属文化企业中唯一的上市公司，公司得到省内各级党委、政府的大力支持，近年来在争取省内各级信息化项目方面具有一定优势。\n\n3. 开放融合优势：公司通过吸引合作伙伴，共同试点新技术、探索新业态、布局新业务，打造合作生态圈，降低经营风险，提高转型质量。\n\n4. 本地化优势：公司立足陕西当地，提供本地化服务，通过打造智慧酒店、智慧美丽乡村等智慧门户，形成差异化、本地化、定制化的内容，成为区别于其他服务商的突出优势。"

question: 浙商证券股份有限公司2021年利息支出是多少元?
error: _log() got an unexpected keyword argument 'exc_info'
question: 记账本位币是什么意思？
question_classification: {'company': '', 'year': [], 'keyword': ['记账本位币'], 'formula': False, 'type': '3-2'}
question_answer: " 记账本位币是指一个国家或地区在记录和报告经济活动时所采用的货币。这个货币通常是该国或地区的官方货币，也可能是其他货币，比如在某些情况下，跨国公司可能会使用一种共同的本位币来记录其全球业务。\n\n记账本位币用于编制各种经济账目和财务报表，包括国家的 GDP、通货膨胀率、贸易顺差和外债等。采用记账本位币可以方便国家或地区之间的经济比较和统计分析，同时也可以更好地反映一个国家或地区的经济状况和趋势。"

question: 2019年盐田港企业的硕士及以上人员占职工人数比例是多少？保留两位小数。
question_classification: {'company': '盐田港企业', 'year': [2019], 'keyword': ['硕士及以上人员', '职工人数', '比例'], 'formula': True, 'type': '1'}
question_sql:

d:\anaconda3\lib\site-packages\langchain\schema\vectorstore.py:259: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='（8）主要销售客户和主要供应商情况', metadata={'id': 59}), -149.5675109857921)]
  warnings.warn(
d:\anaconda3\lib\site-packages\langchain\schema\vectorstore.py:259: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='（1）以客户为中心，超越客户期望。明确以客户为中心、以业务聚焦发展为指导思想，树立进一步提升外部客户关系、强化内部客户关系、聚焦细分行业做强的高质量发展思路；', metadata={'id': 97}), -162.63740920421947)]
  warnings.warn(


question_answer: " 根据 2019 年红宝丽的年报，公司前五名客户合计销售金额为 950,131,646.57 元，占年度销售总额比例为 39.87%。这一客户集中度属于中等水平，说明公司在销售方面并不依赖于少数几个大客户，销售风险相对较低。在同行业中，这一客户集中度可能处于中等偏上水平，具体情况还需结合其他上市公司年报进行分析。"

question: 2019年乐普医疗企业研发经费占费用比例是多少?保留2位小数。
question_classification: {'company': '乐普医疗', 'year': [2019], 'keyword': ['研发经费', '费用比例'], 'formula': True, 'type': '2-1'}
question_sql: select 公司名称,股票简称,报告年份,研发费用/（研发费用 + 管理费用）*100 as 研发经费占比 from company where 报告年份 in ('2019') and (公司名称 in ('乐普医疗') or 股票简称 in ('乐普医疗'));
error: no such column: （研发费用
question: 2020年东风股份营业收入是多少元?
question_classification: {'company': '东风股份', 'year': [2020], 'keyword': ['营业收入'], 'formula': False, 'type': '1'}
question_sql: select 公司名称,股票简称,报告年份,营业收入 from company where 报告年份 in ('2020') and (公司名称 in ('东风股份') or 股票简称 in ('东风股份'));
json_data: [
    {
        "公司名称": "汕头东风印刷股份有限公司",
        "股票简称": "东风股份",
        "报告年份": 2020,
        "营业收入": "3,069,393,914.54"
    }
]
question_answer: 2020 年东风股份的营业收入为 3,069,393,914.54 元。

question: 请问，富临运业在2021

IndentationError: expected an indented block (4020197248.py, line 3)

In [39]:
line_list = []
with open(target_file_path, "r", encoding="utf-8") as file:
    for line in file.readlines():
        line_list.append(json.loads(line))
with open(target_file_path, "w", encoding="utf-8") as file:
    for line in line_list:
        line["answer"] = line["answer"].replace(',', '')
        file.write(json.dumps(line, ensure_ascii=False) + "\n")
